In [7]:
import socket
from cryptography.fernet import Fernet

# Gera uma chave para criptografia/descriptografia
key = Fernet.generate_key()
cipher = Fernet(key)

# Configurações
proxy_host = 'localhost'
proxy_port = 8888
server_host = 'localhost'
server_port = 9999

def final_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((server_host, server_port))
        server_socket.listen(1)
        print(f"Final server listening on {server_host}:{server_port}")

        while True:
            proxy_socket, proxy_address = server_socket.accept()
            with proxy_socket:
                print(f"Connection from {proxy_address}")
                data = proxy_socket.recv(1024)
                print(f"Received: {data.decode()}")
                response = b"Dados recebidos com sucesso"
                proxy_socket.sendall(response)

def proxy_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as proxy_socket:
        proxy_socket.bind((proxy_host, proxy_port))
        proxy_socket.listen(1)
        print(f"Proxy server listening on {proxy_host}:{proxy_port}")

        while True:
            client_socket, client_address = proxy_socket.accept()
            with client_socket:
                print(f"Connection from {client_address}")
                encrypted_data = client_socket.recv(1024)
                decrypted_data = cipher.decrypt(encrypted_data)

                # Conectar ao servidor final
                with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
                    server_socket.connect((server_host, server_port))
                    server_socket.sendall(decrypted_data)
                    response = server_socket.recv(1024)

                encrypted_response = cipher.encrypt(response)
                client_socket.sendall(encrypted_response)

def client():
    data = b"Mensagem secreta"
    encrypted_data = cipher.encrypt(data)

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        client_socket.connect((proxy_host, proxy_port))
        client_socket.sendall(encrypted_data)
        encrypted_response = client_socket.recv(1024)
        decrypted_response = cipher.decrypt(encrypted_response)
        print(f"Response from server: {decrypted_response.decode()}")

if __name__ == "_main_":
    from multiprocessing import Process

    server_process = Process(target=final_server)
    proxy_process = Process(target=proxy_server)
    client_process = Process(target=client)

    server_process.start()
    proxy_process.start()

    # Espera um pouco para garantir que os servidores estejam prontos
    import time
    time.sleep(1)

    client_process.start()

    client_process.join()
    proxy_process.terminate()
    server_process.terminate()

In [9]:
import socket
from cryptography.fernet import Fernet

# Gera uma chave para criptografia/descriptografia
key = Fernet.generate_key()
cipher = Fernet(key)

# Configurações
proxy_host = 'localhost'
proxy_port = 8888
server_host = 'localhost'
server_port = 9999

def final_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((server_host, server_port))
        server_socket.listen(1)
        print(f"Final server listening on {server_host}:{server_port}")

        while True:
            proxy_socket, proxy_address = server_socket.accept()
            with proxy_socket:
                print(f"Connection from {proxy_address}")
                data = proxy_socket.recv(1024)
                print(f"Received: {data.decode()}")
                response = b"Dados recebidos com sucesso"
                proxy_socket.sendall(response)

def proxy_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as proxy_socket:
        proxy_socket.bind((proxy_host, proxy_port))
        proxy_socket.listen(1)
        print(f"Proxy server listening on {proxy_host}:{proxy_port}")

        while True:
            client_socket, client_address = proxy_socket.accept()
            with client_socket:
                print(f"Connection from {client_address}")
                encrypted_data = client_socket.recv(1024)
                decrypted_data = cipher.decrypt(encrypted_data)

                # Conectar ao servidor final
                with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
                    server_socket.connect((server_host, server_port))
                    server_socket.sendall(decrypted_data)
                    response = server_socket.recv(1024)

                encrypted_response = cipher.encrypt(response)
                client_socket.sendall(encrypted_response)

def client():
    data = b"Mensagem secreta"
    encrypted_data = cipher.encrypt(data)

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        client_socket.connect((proxy_host, proxy_port))
        client_socket.sendall(encrypted_data)
        encrypted_response = client_socket.recv(1024)
        decrypted_response = cipher.decrypt(encrypted_response)
        print(f"Response from server: {decrypted_response.decode()}")

if __name__ == "__main__":
    from multiprocessing import Process

    server_process = Process(target=final_server)
    proxy_process = Process(target=proxy_server)
    client_process = Process(target=client)

    server_process.start()
    proxy_process.start()

    # Espera um pouco para garantir que os servidores estejam prontos
    import time
    time.sleep(1)

    client_process.start()

    client_process.join()
    proxy_process.terminate()
    server_process.terminate()

Final server listening on localhost:9999Proxy server listening on localhost:8888

Connection from ('127.0.0.1', 60442)
Connection from ('127.0.0.1', 58804)
Received: Mensagem secreta
Response from server: Dados recebidos com sucesso
